In [19]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [20]:
# 1. Load raw CSVs
df1 = pd.read_csv('CIS62.20250516213458.csv')  # IP engagement rates 
df2 = pd.read_csv('BSA02.20250516T100541.csv')  # Total R&D expenditure 
df3 = pd.read_csv('BSA22.20250516T200531.csv')  # R&D enterprise headcounts 

In [32]:
# 2. Define a simple inspection function
def inspect_df(df, name):
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)                     # rows, columns
    print("\nInfo:")
    df.info()                                     # schema and non-null counts
    print("\nNull values per column:")
    print(df.isnull().sum())                      # count of missing values
    print("\nDescriptive statistics (numeric):")
    print(df.describe(), "\n")                    # count, mean, std, min/max, quartiles
    print("First 5 rows:")
    print(df.head(), "\n")                        # preview

In [22]:
inspect_df(df1, 'df1 (CIS62: IP engagement rates)')
inspect_df(df2, 'df2 (BSA02: R&D expenditure)')
inspect_df(df3, 'df3 (BSA22: R&D enterprise counts)')


=== df1 (CIS62: IP engagement rates) ===
Shape: (105, 10)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   STATISTIC           105 non-null    object 
 1   STATISTIC Label     105 non-null    object 
 2   TLIST(A1)           105 non-null    int64  
 3   Year                105 non-null    int64  
 4   C03337V04024        105 non-null    int64  
 5   Type of Innovation  105 non-null    object 
 6   C02673V03235        105 non-null    object 
 7   NACE Rev 2 Sector   105 non-null    object 
 8   UNIT                105 non-null    object 
 9   VALUE               75 non-null     float64
dtypes: float64(1), int64(3), object(6)
memory usage: 8.3+ KB

Null values per column:
STATISTIC              0
STATISTIC Label        0
TLIST(A1)              0
Year                   0
C03337V04024           0
Type of Innovation     0
C026

In [24]:
# 1.2 Pivot df1: one column per (Type of Innovation, Sector)
df1_wide = (
    df1
    .assign(VALUE=lambda d: pd.to_numeric(d['VALUE'], errors='coerce'))
    .pivot_table(
        index='Year',
        columns=['Type of Innovation','NACE Rev 2 Sector'],
        values='VALUE'
    )
    .rename_axis(columns=['innovation_type','sector'])
)

# 1.3 Pivot df2: one column per (Statistic Label, Ownership)
df2_wide = (
    df2
    .assign(VALUE=lambda d: pd.to_numeric(d['VALUE'], errors='coerce'))
    .pivot_table(
        index='Year',
        columns=['Statistic Label','Nationality of Ownership'],
        values='VALUE'
    )
    .rename_axis(columns=['statistic','ownership'])
)

# 1.4 Pivot df3: one column per (Expenditure band, Ownership)
df3_wide = (
    df3
    .assign(VALUE=lambda d: pd.to_numeric(d['VALUE'], errors='coerce'))
    .pivot_table(
        index='Year',
        columns=['Expenditure','Nationality of Ownership'],
        values='VALUE'
    )
    .rename_axis(columns=['expenditure_band','ownership'])
)

In [33]:
# 3. Merge into a single DataFrame called df
df = df1_wide.join(df2_wide, how='outer').join(df3_wide, how='outer')
df.index.name = 'Year'
df.reset_index(inplace=True)

In [34]:
df.shape

(18, 100)

In [35]:
df.head()

Year All enterprises claiming copyright  \
                     Industries (05 to 39)   
0  2007                                NaN   
1  2008                                NaN   
2  2009                                NaN   
3  2010                                NaN   
4  2011                                NaN   

                                                                                      \
  Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73)   
0                                                NaN                                   
1                                                NaN                                   
2                                                NaN                                   
3                                                NaN                                   
4                                                NaN                                   

                                                      \
  Selected Services (46, 49-53, 58-63, 64-66, 71-73)   
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

  All enterprises using trade secrets  \
                Industries (05 to 39)   
0                                 NaN   
1                                 NaN   
2                                 NaN   
3                                 NaN   
4                                 NaN   

                                                                                      \
  Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73)   
0                                                NaN                                   
1                                                NaN                                   
2                                                NaN                                   
3                                                NaN                                   
4                                                NaN                                   

                                                      \
  Selected Services (46, 49-53, 58-63, 64-66, 71-73)   
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

  Any intellectual innovation  \
        Industries (05 to 39)   
0                         NaN   
1                         NaN   
2                         NaN   
3                         NaN   
4                         NaN   

                                                                                      \
  Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73)   
0                                                NaN                                   
1                                                NaN                                   
2                                                NaN                                   
3                                                NaN                                   
4                                                NaN                                   

                                                      ... €100,000 - €499,999  \
  Selected Services (46, 49-53, 58-63, 64-66, 71-73)  ... Non Irish ownership   
0                                                NaN  ...               66.00   
1                                                NaN  ...                 NaN   
2                                                NaN  ...               56.65   
3                                                NaN  ...                 NaN   
4                                    

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 100 columns):
 #   Column                                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                                             --------------  -----  
 0   (Year, )                                                                                                                           18 non-null     int64  
 1   (All enterprises claiming copyright, Industries (05 to 39))                                                                        4 non-null      float64
 2   (All enterprises claiming copyright, Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73))           4 non-null      float64
 3   (All enterprises claiming copyright, Selected Services (46, 49-53, 58-63, 64-66, 71-73))   

In [37]:
df.describe()

Year All enterprises claiming copyright  \
                                Industries (05 to 39)   
count    18.000000                           4.000000   
mean   2015.500000                           2.675000   
std       5.338539                           1.512999   
min    2007.000000                           1.000000   
25%    2011.250000                           1.750000   
50%    2015.500000                           2.600000   
75%    2019.750000                           3.525000   
max    2024.000000                           4.500000   

                                                                                          \
      Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73)   
count                                            4.00000                                   
mean                                             2.80000                                   
std                                              1.00995                                   
min                                              1.30000                                   
25%                                              2.65000                                   
50%                                              3.25000                                   
75%                                              3.40000                                   
max                                              3.40000                                   

                                                          \
      Selected Services (46, 49-53, 58-63, 64-66, 71-73)   
count                                           4.000000   
mean                                            2.825000   
std                                             1.014479   
min                                             1.400000   
25%                                             2.450000   
50%                                             3.150000   
75%                                             3.525000   
max                                             3.600000   

      All enterprises using trade secrets  \
                    Industries (05 to 39)   
count                            4.000000   
mean                             5.025000   
std                              1.878608   
min                              2.700000   
25%                              4.425000   
50%                              5.050000   
75%                              5.650000   
max                              7.300000   

                                                                                          \
      Industries and selected services (05 to 39,46,49 to 53,58 to 63,64 to 66,71 to 73)   
count                                           4.000000                                   
mean                                            3.650000                                   
std                                             1.436431                                   
min                                             2.100000                                   
25%                                             2.850000                                   
50%                                             3.500000                                   
75%                                             4.300000                                   
max                                             5.500000                                   

                                                          \
      Selected Services (46, 49-53, 58-63, 64-66, 71-73)   
count                                           4.000000   
mean                                            2.925000   
std                                             1.233896   
min                                             1.800000   
25%                                             2.025000   
50%                                             2.700000   
75%                                             3.600000   
max      

In [38]:
import pandas as pd

# Assume df_clean is the DataFrame you just described (18×100)
df = df_clean.copy()

# 1. Compute non-null counts and proportions
non_null_counts = df.notna().sum()
coverage = non_null_counts / len(df)

summary = pd.DataFrame({
    'non_null': non_null_counts,
    'coverage_pct': coverage * 100
}).sort_values('coverage_pct')

print("Columns sorted by coverage (bottom 10):")
print(summary.head(10), "\n")

# 2. Drop columns with coverage below 60%
threshold = 0.6
cols_to_keep = summary[summary['coverage_pct'] >= threshold * 100].index
df_reduced = df[cols_to_keep].copy()

print(f"Dropped {len(df.columns) - len(cols_to_keep)} columns; retained {len(cols_to_keep)} features.")
print("New shape:", df_reduced.shape)

# 3. Quick check of remaining coverage
print("\nMinimum coverage among retained features:",
      coverage[cols_to_keep].min() * 100, "%")

Columns sorted by coverage (bottom 10):
                                                    non_null  coverage_pct
year                                                      18         100.0
estimated_current_expenditure_other_current_cos...        18         100.0
estimated_current_expenditure_other_current_cos...        18         100.0
estimated_current_expenditure_other_current_cos...        18         100.0
estimated_current_expenditure_labour_costs_non_...        18         100.0
estimated_current_expenditure_labour_costs_iris...        18         100.0
estimated_current_expenditure_labour_costs_all_...        18         100.0
estimated_capital_expenditure_payments_made_for...        18         100.0
estimated_capital_expenditure_payments_made_for...        18         100.0
estimated_capital_expenditure_payments_made_for...        18         100.0 

Dropped 0 columns; retained 97 features.
New shape: (18, 97)

Minimum coverage among retained features: 100.0 %
